In [1]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tradingview_ta import Interval

# Añadir la ruta base del proyecto a sys.path
base_path = os.path.abspath('..')
if base_path not in sys.path:
    sys.path.append(base_path)

# Importar librerías
try:
    from src.data.data_loader import load_data
    from src.utils.logger import Logger
    print("Importaciones realizadas correctamente")
except ModuleNotFoundError as e:
    print(f"Error en las importaciones: {e}")
    raise

# Verificar rutas de configuración y datos de mercado
config_path = os.path.abspath('../config.json')

if not os.path.exists(config_path):
    print(f"El archivo de configuración no existe: {config_path}")

# Definir fechas de inicio y fin
start_date = pd.to_datetime('2025-01-01')
end_date = pd.to_datetime('2025-01-13')

# Cargar datos de mercado
try:
    logger = Logger(__name__)
    market_data = load_data(symbol="EURUSD", exchange="OANDA", interval=Interval.INTERVAL_1_HOUR, start_date=start_date, end_date=end_date, source="finnhub")
    if market_data is not None:
        print("Datos de mercado cargados correctamente.")
        print(market_data.head())

        # Visualizar el gráfico
        plt.figure(figsize=(12, 6))
        plt.plot(market_data.index, market_data['close'], label='Close Price')
        plt.title('EUR/USD Price Chart')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.show()
    else:
        logger.error("No se pudieron cargar los datos")

except Exception as e:
    logger.error(f"Error al cargar los datos del mercado: {e}")

Importaciones realizadas correctamente


2025-01-15 17:26:51,768 - __main__ - ERROR - No se pudieron cargar los datos


Error de Finnhub: FinnhubAPIException(status_code: 403): You don't have access to this resource.
